

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/T5TRANSFORMER.ipynb)




# **Text Summarization & Question Answering using google's T5 Transformer**

### Spark NLP documentation and instructions:
https://nlp.johnsnowlabs.com/docs/en/quickstart

### You can find details about Spark NLP annotators here:
https://nlp.johnsnowlabs.com/docs/en/annotators

### You can find details about Spark NLP models here:
https://nlp.johnsnowlabs.com/models


## 1. Colab Setup

In [ ]:
# Install java
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
!java -version

# This is only to setup PySpark and Spark NLP on Colab
# -p is for pyspark
# -s is for spark-nlp
# !bash colab_setup.sh -p 3.1.1 -s 3.0.0  
# by default they are set to the latest

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/colab_setup.sh
!bash colab_setup.sh

# Install Spark NLP Display for visualization
! pip install --upgrade -q spark-nlp-display

## 2. Start the Spark session

Import dependencies and start Spark session.

In [ ]:
import os
import json
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"

import pandas as pd
import numpy as np

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()

## 3. Select the DL model

For complete model list: 
https://nlp.johnsnowlabs.com/models

For `T5` models:
https://nlp.johnsnowlabs.com/models?tag=t5

##4. Text Summaization using T5 Transformer

 Define Spark NLP pipeline

In [ ]:
document_assembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("documents")

t5 = T5Transformer() \
  .pretrained("t5_small", 'en') \
  .setTask("summarize:")\
  .setMaxOutputLength(200)\
  .setInputCols(["documents"]) \
  .setOutputCol("summaries")

summarizer_pp = Pipeline(stages=[
    document_assembler, t5
])

t5_small download started this may take some time.
Approximate size to download 168.7 MB
[OK!]


Run the pipeline

In [ ]:
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = summarizer_pp.fit(empty_df)
sum_lmodel = LightPipeline(pipeline_model)

example_txt = """

Mona Lisa is a 16th century oil painting created by Leonardo. It is held at the Louvre in Paris.
"""

res = sum_lmodel.fullAnnotate(example_txt)[0]


print ('Summary:', res['summaries'][0].result)

Summary: mona Lisa is a 16th century oil painting created by Leonardo .


##5. Question Answering using T5 Transformer

 Define Spark NLP pipeline

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

sentence_detector = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl", "en")\
    .setInputCols(["documents"])\
    .setOutputCol("questions")

t5 = T5Transformer()\
    .pretrained("google_t5_small_ssm_nq", 'en')\
    .setInputCols(["questions"])\
    .setOutputCol("answers")\

qa_pp = Pipeline(stages=[
    document_assembler, sentence_detector, t5
])

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
google_t5_small_ssm_nq download started this may take some time.
Approximate size to download 168.8 MB
[OK!]


Run the pipeline

In [ ]:
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = qa_pp.fit(empty_df)
qa_lmodel = LightPipeline(pipeline_model)

questions = ["What is the capital of America?",
             "who is the most famous singer?",
             "when do we have winters?",
             "Who is the president of the USA?",
             "Who is the founder of Microsoft?"
]

res = qa_lmodel.fullAnnotate(questions)


for i, r in enumerate(res):
  print ("Question:", questions[i])
  for sent in r['answers']:
    print ('Answer:\t', sent.result)


Question: What is the capital of America?
Answer:	 Washington, D.C.
Question: who is the most famous singer?
Answer:	 Selena Gomez
Question: when do we have winters?
Answer:	 February 3
Question: Who is the president of the USA?
Answer:	 Donald Trump
Question: Who is the founder of Microsoft?
Answer:	 Microsoft founder Bill Gates
